# Demo 02: View water column data

## Introduction

This Jupyter Notebook contains an introduction to viewing water column data using Ping.

## Here you will learn

- 

## Short version

At the end of the notebook you will find the important parts of this notebook, compiled into a single cell


## ---

### First: Import the necessary modules

In [1]:
# use widgets as matplotlib backend
%matplotlib widget

# imports
from matplotlib import pyplot as plt
import os
import sys
import numpy as np
from collections import defaultdict
from time import time
from tqdm.auto import tqdm #progress bar, auto selects a backend based on the environment e.g. for a jupyter notebook it will use the widget version

from typing import Union, Tuple

import matplotlib as mpl
import matplotlib.dates as mdates

import datetime as dt

# import the file object for opening kongsberg files
# Note: function and library naming to be discussed
from themachinethatgoesping.echosounders import simradraw,kongsbergall          # This is the filereader module for .all/.wcd files
from themachinethatgoesping.echosounders import filetemplates      # This is the filereader module for .all/.wcd files
from themachinethatgoesping.echosounders import pingtools          # This is the filereader module for .all/.wcd files
from themachinethatgoesping.echosounders import index_functions # This is the module for creating and using indexes to speed up file initialization
import themachinethatgoesping.tools as ptools                   # Some tools for working with ping (here we use timeconv for timestamp conversion)
import themachinethatgoesping.algorithms as palgorithms                   # Some tools for working with ping (here we use timeconv for timestamp conversion)
import themachinethatgoesping.navigation as pnav                   # Some tools for working with ping (here we use timeconv for timestamp conversion)

import themachinethatgoesping.pingprocessing.watercolumn.make_image as mi
import themachinethatgoesping.pingprocessing.watercolumn.helper.make_image_helper as mi_hlp

from themachinethatgoesping.pingprocessing import filter_pings, overview, split_pings, group_pings
from themachinethatgoesping.pingprocessing import watercolumn
gp = palgorithms.geoprocessing
import pandas as pd

# simplify creating figures
mpl.rcParams['figure.dpi'] = 100
close_plots: bool = True

def create_figure(name: str, return_ax: bool = True) -> Union[Tuple[plt.Figure, plt.Axes], plt.Figure]:
    """Helper function to create a figure with a given name that returns the figure and axis
    """
    if close_plots:
        plt.close(name)
    fig = plt.figure(name)
    fig.suptitle = name

    if return_ax:
        return fig, fig.subplots()
    return fig

def clear_memory():
    import gc
    gc.collect()
    
    import ctypes
    libc = ctypes.CDLL("libc.so.6") # clearing cache 
    libc.malloc_trim(0)

In [2]:
from collections import OrderedDict

class fake_tqdm(object):
    def __init__(self, w_prg):
        self.w_prg = w_prg
        
    def __call__(self, list_like):
        self.list_like = list_like
        self.list_iter = iter(list_like)
        self.index = 0
        self.total = len(list_like)
        self.w_prg.max = self.total
        self.w_prg.value = 0
        return self
        
    def __iter__(self):
        return self
    
    def __next__(self):
        self.index += 1
        self.w_prg.value = self.index
        return next(self.list_iter)
    
    def __len__(self):
        return self.total
    
    def update(self):
        self.index += 1
        next(self.list_iter)
        self.w_prg.value = self.index
        
    def close(self):
        pass

### Find files to open

This is a simple python loop to find .all and .wcd files in the specified directories

In [3]:
# Define the folder(s) to search for Kongsberg files
# Note: subdirectories will be searched as well
folders = []
#folders.append("./")
#folders.append("/home/users/data/koen campaign 2/")
folders.append("/home/data/turbeams/TURBEAMS 2023-12/")

# Initialize lists for storing file paths and names
files = index_functions.find_files(folders, [".all","wcd"])
files.sort()

file_cache_paths = index_functions.get_cache_file_paths(file_paths=files)

index_functions.remove_name_from_cache(file_cache_paths, "FilePackageCache<RuntimeParameters>")
index_functions.print_cache_file_statistics(file_cache_paths)

Found 122 files


 19%|#8        | 23/122 [00:01<00:04, 22.48it/s]

FilePackageIndex: 192.38 'MB' / 0.45 %
NavigationInterpolatorLatLon: 1654.93 'MB' / 3.84 %
- Combined -: 1847.31 'MB' / 4.29 %
- Source files -: 43059.28 'MB' / 100.0 %


## Open files


In [ ]:
file_cache_paths = index_functions.get_cache_file_paths(file_paths=files)

fm = kongsbergall.KongsbergAllFileHandler_mapped(files[:], file_cache_paths=file_cache_paths, init = True)

Error message: 'failed mapping file: Invalid argument: iostream error'


Error message: 'failed mapping file: Invalid argument: iostream error'


indexing files ⠁ 99% :02s<00m:00s] [Found: 7717334 datagrams in 122 files (43059MB)]                                                 
Initializing navigation ⢀ 92% :05s<00m:00s] [60/63]                    


In [ ]:
all_pings = fm.pings()
all_pings = filter_pings.by_features(all_pings, ["watercolumn"])
print(len(pings))

In [ ]:
fig_wci, ax_wci = create_figure("wci")

In [ ]:
from ipywidgets import *
import ipywidgets as ipw
from time import time

#pings, transducers = group_dual_pings(fm.pings().get_sorted_by_time())

fig_wci.set_tight_layout(True)

#@widgets.interact
#@debounce(0.1)
def update(w):  
    try:
        if w_protect_stack.value:
            if w['owner'] != w_wci_stack:
                if float(w_text_execution_time.value) > 0.5:
                    w_wci_stack.value = w_wci_stack.value * 0.5 / float(w_text_execution_time.value)
        if w_wci_stack.value > 1:
            w_wci.step = int(w_wci_stack.value/2)
        if w_wci_step.value > 0:
            w_wci.step = w_wci_step.value
    except Exception as e:
        pass
    
    
    w_text_num_total.value = str(int(w_text_num_total.value) +1)
    w_text_num_active.value = str(int(w_text_num_active.value) +1)
    
    t = time()
    global a, lax_wci, fig_wci, ping1, ping2, wci, wci1
    a = w
    #print(w)
    wci_index = w_wci.value
    wci_stack = w_wci_stack.value
    wci_stack_step = w_wci_stack_step.value
    cmin = w_cmin.value
    cmax = w_cmax.value
    aspect = w_aspect.value
    hsize = w_hsize.value
    heads = w_heads.value
    interpolation = w_interpolation.value
    maxz = w_z.value
    from_bottom = w_from_bottom.value
    threshold_white = w_threshold.value
    linear_mean = w_linear_stack.value
     
    ping = all_pings[wci_index]
    
    wci_time.set_trait('value',ptools.timeconv.unixtime_to_datestring(ping.get_timestamp(),format='%d-%m-%Y %H:%M:%S'))
    
    if wci_stack > 1:
        max_index = wci_index+wci_stack
        if max_index > len(pings):
            max_index = len(pings)
        pings = []
        for pp in pings[wci_index:max_index:wci_stack_step]:
            for p in pp.values():
                pings.append(p)
                
        #pings = pings[::2]
    

    try:
        if wci_stack > 1:
            wci,extent = mi.make_wci_stack(pings,hsize,progress_bar=progress_bar,linear_mean=linear_mean,from_bottom_xyz=from_bottom, compute_av=w_av.value)
        else:
            if heads == 'split_dual_rect':
                if w_av.value:
                    wci = ping.watercolumn.get_av()
                else:
                    wci = ping1.watercolumn.get_amplitudes()
                extent = [0, ping.watercolumn.get_number_of_beams(),0, ping.watercolumn.get_number_of_samples_per_beam()[0]]
            else:
                wci,extent = mi.make_wci(ping,hsize,from_bottom_xyz=from_bottom)
            
        ax_wci.clear()
        if threshold_white:
            wci[wci < cmin] = np.nan
            
        mapable = ax_wci.imshow(wci.transpose(),aspect=aspect, extent = extent, cmap='YlGnBu_r',vmin=cmin,vmax=cmax, interpolation=interpolation)
        
        if not maxz == -1:
            ax_wci.set_ylim(maxz,0)
                               
        w_text_num_active.value = str(int(w_text_num_active.value) -1)
        w_text_execution_time.value = str(round(time()-t,3))
            
        
    except Exception as e:
        print(e)
        #pass
        raise (e)


w_z = FloatSlider(min=-1, max=50, step=1, value = -1)
w_cmin = FloatSlider(min=-150, max=150, step=5, value = -90)
w_cmax = FloatSlider(min=-150, max=150, step=5, value = -20)
w_wci = IntSlider(min=0, max=len(pings)-1, step=1, value =0)
w_hsize = IntSlider(min=1, max=2048, step=1, value = 1024)

w_from_bottom = Checkbox(description="from bottom", value=False)
w_linear_stack = Checkbox(description="linear stack", value=True)
w_protect_stack = Checkbox(description="protect stacking time", value=False)
w_threshold = Checkbox(description="threshhold white", value=False)
w_av = Checkbox(description="compute AV", value=True)

w_aspect = Dropdown(options=['auto', 'equal'], value='equal')
w_heads = Dropdown(options=['blend_dual', 'blend_dual_inverse', 'split_dual', 'split_dual_rect'], value='blend_dual')
w_interpolation = Dropdown(options=['antialiased', 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman'], value='nearest')
w_wci_stack = IntText(
    value=1,
    description='stack:',
    disabled=False
)
w_wci_stack_step = IntText(
    value=1,
    description='stack step:',
    disabled=False
)
w_wci_step = IntText(
    value=1,
    description='ping step:',
    disabled=False
)

w_text_num_total = Text(
    value='0',
    placeholder='0',
    description='Total executions:',
    disabled=False   
)
w_text_num_active = Text(
    value='0',
    placeholder='0',
    description='Active executions:',
    disabled=False   
)
w_text_execution_time = Text(
    value='0',
    placeholder='0',
    description='Time of last execution:',
    disabled=False   
)
w_progress = IntProgress(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Stacking:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)
progress_bar = fake_tqdm(w_progress)

wci_time = ipw.Label()

box_ping_slider = HBox([w_wci, wci_time])
box_text = HBox([w_text_num_total,w_text_num_active,w_text_execution_time])
box_options = HBox([w_aspect,w_heads,w_interpolation,w_wci_stack,w_wci_stack_step,w_wci_step])
box_check = HBox([w_from_bottom,w_linear_stack,w_protect_stack, w_threshold, w_av])

w_z.observe(update, names=['value'])
w_cmin.observe(update, names=['value'])
w_cmax.observe(update, names=['value'])
w_wci.observe(update, names=['value'])
w_wci_stack.observe(update, names=['value'])
w_wci_stack_step.observe(update, names=['value'])
w_wci_step.observe(update, names=['value'])
w_hsize.observe(update, names=['value'])
w_aspect.observe(update, names=['value'])
w_from_bottom.observe(update, names=['value'])
w_threshold.observe(update, names=['value'])
w_av.observe(update, names=['value'])
w_linear_stack.observe(update, names=['value'])
w_heads.observe(update, names=['value'])
w_interpolation.observe(update, names=['value'])


update(0)
display(fig_wci.canvas,w_progress, box_text, box_options, box_check, w_z, w_cmin,w_cmax,box_ping_slider,w_hsize)

In [ ]:
pings